In [1]:
import os
import sys

sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))
from pathlib import Path

import matplotlib.pyplot as plt
from pydicom import dcmread
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

from src.utils.imageProcessing import *
from src.utils.utils import *

# Data MongoDB

## Gerar os inserts

CBIS-DDSM

In [2]:
# Cria o objeto do banco de dados
database_CBIS_DDSM = {
            "name": "CBIS-DDSM",
            "exameType": "Mamografia",
            "description": "Banco de dados gerado pelo processamento do banco DDSM",
            "imageQuality": [],
            "imageType": "DICOM",
            "sourceLink": "https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=22516629",
}

studies = []
images = []
path_metadata = "../../outputs/mamografia/extracted_metadata"

list_metadata = load_json("extracted_metadata_CBIS-DDSM", path_metadata)
    
for i, metadata in enumerate(list_metadata):
    metadata_csv = rename_keys(metadata["metadata_csv"])
    metadata_dicom = metadata["metadata_dicom_files"]["original"][0]
    
    database_CBIS_DDSM["imageQuality"].append(int(metadata_dicom["bits_stored (0028, 0101)"]))
    resizedImagePath = f"D:\\mathe\\Documents\BancoDados_IC\\resized_images\\CBIS-DDSM\\image_resized_CBIS-DDSM_{i}.png"
    
    directory = Path(metadata_csv["original_image_path"])
    images_paths = list(directory.rglob(f"*.dcm*"))
    images_paths = list(map(str, images_paths))
    metadata_csv["original_image_path"] = images_paths[0]
    
    """ image_array = extract_image_dicom(metadata_csv["original_image_path"])
    resize_image(image_array, (255, 255), save=True, path=resizedImagePath, image_type="png") """
    
    image = {
             "patient_id": metadata_csv["patient_id"],
             "leftOrRightBreast": metadata_csv["left_or_right_breast"],
             "orientation": metadata_csv["image_view"],
             "width": metadata_dicom["columns (0028, 0011)"],
             "height": metadata_dicom["rows (0028, 0010)"],
             "imagePath": metadata_csv["original_image_path"],
             "resizedImagePath": resizedImagePath,
             "imageQuality": metadata_dicom["bits_stored (0028, 0101)"],
             "classification": {
                                 "pathology": metadata_csv["pathology"],
                                 "biRads": metadata_csv["bi_rads"],
                                 "breastDensity": metadata_csv["breast_density"],
                                 "abnormalityType": metadata_csv["abnormality_type"],
                                 "subtlety": metadata_csv["subtlety"]
                                },
            }

    images.append(image)

database_CBIS_DDSM["imageQuality"] = list(set(database_CBIS_DDSM["imageQuality"]))

if len(database_CBIS_DDSM["imageQuality"]) == 1:
       database_CBIS_DDSM["imageQuality"] = database_CBIS_DDSM["imageQuality"][0]
    
database_CBIS_DDSM["images"] = images

CMMD

In [3]:
# Cria o objeto do banco de dados
database_CMMD = {
            "name": "CMMD",
            "exameType": "Mamografia",
            "description": "Banco de dados CMMD",
            "imageQuality": [],
            "imageType": "Dicom",
            "sourceLink": "https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=70230508",
}

images = []

# Carrega os metadados
json_name = 'csv_and_dicom_CMMD'
list_metadata = load_json("extracted_metadata_CMMD", "../../outputs/mamografia/extracted_metadata")

for i, metadata in enumerate(list_metadata):
    metadata_csv = rename_keys(metadata["metadata_csv"])
    
    n = [j for j in range(len(metadata_csv['image_path']))] 
    for j, image_path, metadata_dicom in zip(n, metadata_csv['image_path'], metadata["metadata_dicom_files"]):
        
        resizedImagePath = f"D:\\mathe\\Documents\BancoDados_IC\\resized_images\\CMMD\\image_resized_CMMD_{i}-{j}.png"
        database_CMMD["imageQuality"].append(int(metadata_dicom["bits_stored (0028, 0101)"]))
        
        image_array = extract_image_dicom(image_path)
        resize_image(image_array, (255, 255), save=True, path=resizedImagePath, image_type="png")
        
        image = {
             "patient_id": metadata_csv["patient_id"],
             "patient_age": metadata_csv["patient_age"],
             "leftOrRightBreast": metadata_csv["left_or_right_breast"],
             "width": metadata_dicom["columns (0028, 0011)"],
             "height": metadata_dicom["rows (0028, 0010)"],
             "imagePath": image_path,
             "resizedImagePath": resizedImagePath,
             "imageQuality": metadata_dicom["bits_stored (0028, 0101)"],
             "classification": {
                                 "pathology": metadata_csv["pathology"],
                                 "abnormalityType": metadata_csv["abnormality_type"],
                                 "subtype": metadata_csv["subtype"]
                                },
            }
        
        if metadata_csv['subtype'] == 'NaN':
            del image["classification"]["subtype"]
            
        images.append(image)
 
database_CMMD["imageQuality"] = list(set(database_CMMD["imageQuality"]))
    
if len(database_CMMD["imageQuality"]) == 1:
       database_CMMD["imageQuality"] = database_CMMD["imageQuality"][0]
    
database_CMMD["images"] = images

## Salvar no banco de dados

In [5]:
def get_db(database_name, host:str ="localhost", port:int =27017): 
    """Connecto to mongodb"""
    try:
        connection = MongoClient(host, port)
        return connection[database_name]
    except ConnectionFailure:
        sys.stderr.write("Could not connect to MongoDB")
        sys.exit(1)
              

""" db = get_db('radiomica')
metadata = db.metadata """

databases = [database_CBIS_DDSM, database_CMMD]

save_json("databases", databases, "../../outputs/mamografia") 
# metadata.insert_many(databases)
len(database_CMMD["images"])

5590